## Import the Model

In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

## Instantiate Model

In [38]:

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## Data Collection

In [39]:
r = requests.get('https://www.imdb.com/title/tt6443346/reviews?dir=asc&ratingFilter=0')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*text show-more__control.*')
results = soup.find_all('div', {'class':regex})
reviews = [result.text for result in results]

## Load Reviews

In [40]:
reviews

["Just saw the black adam and had a great experience. Critics are judging this movie by the marvel standards with perfect ethics and first world problem. This is the first superhero movie I saw ehich shed a small light on the third world countries. Critics didn't relate to the problems of the third world countries so they have different understanding of a superhero. But superhero is not the one who doesn't kill but the one who understand what needs to be done. This is the perfect example of how DC universe should be build upon.I know it didn't had much character building done in the story. Although it showed alot more than that, that is, how to build a cenematic universe.",
 "Once I start watching a movie I almost always watch it to the end. However with Black Adam I was soo disappointed that I walked out after 40 minutes. Such a disappointment, I really like the Rock, but this movie was just all about the CGI. Poor acting, poor script and story line, ideas stolen from too many movies,

In [41]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [42]:
df.head()

,review
0,Just saw the black adam and had a great experi...
1,Once I start watching a movie I almost always ...
2,The movie isn't a bad movie the action sequenc...
3,The movie is a great time and definitely recom...
4,This movie was so continuous. I never felt lik...


In [43]:
df['review'].iloc[0]

"Just saw the black adam and had a great experience. Critics are judging this movie by the marvel standards with perfect ethics and first world problem. This is the first superhero movie I saw ehich shed a small light on the third world countries. Critics didn't relate to the problems of the third world countries so they have different understanding of a superhero. But superhero is not the one who doesn't kill but the one who understand what needs to be done. This is the perfect example of how DC universe should be build upon.I know it didn't had much character building done in the story. Although it showed alot more than that, that is, how to build a cenematic universe."

## Sentiment Score

In [44]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [45]:
sentiment_score(df['review'].iloc[1])

2

In [46]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [47]:
df

,review,sentiment
0,Just saw the black adam and had a great experi...,5
1,Once I start watching a movie I almost always ...,1
2,The movie isn't a bad movie the action sequenc...,3
3,The movie is a great time and definitely recom...,5
4,This movie was so continuous. I never felt lik...,4
5,"Black Adam is a full on Cinamatic experience, ...",5
6,There are always the same mistakes. It's reall...,2
7,"I set my bar low, so I wont be neither dissapo...",2
8,I watched this movie today. And (despite the f...,4
9,Black Adam is competently made. It handles set...,3


In [48]:
df['review'].iloc[3]

"The movie is a great time and definitely recommend! The cgi could have been better and they cold have shown how strong Doctor fate was better. Doctor Fate is much stronger then they portrayed him to be. I think this is a great installment to the DCEU. I'm truly want to see more of Hawkman and Dr. Fate. No I need to see more of them. Would love to see them build up the justice society. Loved The justice society. Everything aft this point will be a repeat because I need a max of 600 characters to post this, just a heads up. The movie is a great time and definitely recommend! The cgi could have been better and they cold have shown how strong Doctor fate was better."